In [2]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 26.3 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### 데이터 불러오기

In [5]:
import pandas as pd

df_agoda = pd.read_csv('/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/긍부정라벨링/df_agoda', encoding='utf-8')
df_airbnb = pd.read_csv('/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/긍부정라벨링/df_airbnb', encoding='utf-8')
df_booking = pd.read_csv('/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/긍부정라벨링/df_booking', encoding='utf-8')
df_dailyhotel = pd.read_csv('/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/긍부정라벨링/df_dailyhotel', encoding='utf-8')
df_hotelsdotcom = pd.read_csv('/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/긍부정라벨링/df_hotelsdotcom', encoding='utf-8')
df_myrealtrip = pd.read_csv('/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/긍부정라벨링/df_myrealtrip', encoding='utf-8')
df_yanol = pd.read_csv('/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/긍부정라벨링/df_yanol', encoding='utf-8')
df_yeogi = pd.read_csv('/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/긍부정라벨링/df_yeogi', encoding='utf-8')

In [6]:
all_OTA = pd.concat([df_agoda,df_airbnb,df_booking,df_dailyhotel,df_hotelsdotcom,df_myrealtrip,df_yanol,df_yeogi],ignore_index=True)

all_OTA

,date,content,score,at,sentiment
0,0,사용 후기를 통해 쉽게 예매할 수 있어 좋아요,5,20221231223041,1
1,1,세금 포함 최종 가격 올려주는 기능은 없나요 혼동되네요,4,20221231142125,1
2,2,저렴한 가격 최고의 어플,5,20221231141747,1
3,3,저렴한 가격에 좋은 곳을 소개받을 수 있어 좋아요,5,20221231093122,1
4,4,내용들이 선택하기 좋게 잘 되어 있어요,5,20221231030646,1
...,...,...,...,...,...
25122,3496,너무 조아여 가격 저렴해요,5,20200104093150,1
25123,3497,환불 자체를 아예 안 해주는 편이네 요항상 방 배정이랑 방일 적실 수 환불 전혀 안...,1,20200103215249,0
25124,3498,호텔 예약했는데 알고 보니 입실시간이 저녁 시네요 황당한 네 그기에 당일 예약이라 ...,1,20200103160905,0
25125,3499,다운로드 대기 중이라고만 뜨고 계속 다운이 안 되는데 해결할 방법이 있나요,1,20200102095048,0


In [7]:
all_OTA = all_OTA.drop(columns = ['date','score','at'])

all_OTA

,content,sentiment
0,사용 후기를 통해 쉽게 예매할 수 있어 좋아요,1
1,세금 포함 최종 가격 올려주는 기능은 없나요 혼동되네요,1
2,저렴한 가격 최고의 어플,1
3,저렴한 가격에 좋은 곳을 소개받을 수 있어 좋아요,1
4,내용들이 선택하기 좋게 잘 되어 있어요,1
...,...,...
25122,너무 조아여 가격 저렴해요,1
25123,환불 자체를 아예 안 해주는 편이네 요항상 방 배정이랑 방일 적실 수 환불 전혀 안...,0
25124,호텔 예약했는데 알고 보니 입실시간이 저녁 시네요 황당한 네 그기에 당일 예약이라 ...,0
25125,다운로드 대기 중이라고만 뜨고 계속 다운이 안 되는데 해결할 방법이 있나요,0


In [8]:
print(all_OTA.isnull().values.any())

True


In [9]:
print(all_OTA.isnull().sum())

content      40
sentiment     0
dtype: int64


In [10]:
all_OTA = all_OTA.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(all_OTA.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [11]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()

In [12]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# 한글 형태소 분석기
okt = Okt()

# 형태소 분석 후 불용어 처리
corpus = []
for doc in all_OTA["content"]:
    tagged = okt.pos(doc)
    doc = [word for word, pos in tagged if pos in ['Noun', 'Adjective', 'Adverb']]
    doc = [word for word in doc if word not in stopwords]
    corpus.append(" ".join(doc))


# 단어장 생성
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
{'사용': 5239, '후기': 12337, '통해': 11151, '예매': 7646, '있어': 8954, '좋아요': 9922, '세금': 5601, '포함': 11574, '최종': 10666, '가격': 3, '기능': 1472, '없나요': 7289, '혼동': 12167, '저렴한': 9308, '최고': 10646, '어플': 7190, '좋은': 9981, '소개': 5659, '내용': 1924, '선택': 5511, '좋게': 9869, '있어요': 8962, '최악': 10661, '카드': 10870, '결제': 660, '완료': 7796, '기술': 1519, '문제': 3866, '있다는': 8924, '메시지': 3633, '이런': 8466, '경우': 690, '정상': 9621, '아닌가요': 6577, '여행': 7502, '가기': 11, '정말': 9609, '짜증': 10379, '피해': 11699, '보지': 4454, '조치': 9826, '우리': 7998, '그런': 1313, '아닌': 6575, '같은데': 394, '진행': 10338, '안되는': 6754, '관계': 942, '투척': 11178, '호텔': 12150, '리뷰': 3189, '장난': 9183, '요금': 7860, '얼마': 7227, '캐시': 10896, '만원': 3317, '진짜': 10335, '예약': 7662, '환불': 12260, '불가': 4686, '안내': 6738, '장바구니': 9195, '여러': 7463, '그냥': 1252, '바로': 4009, '확인': 12250, '없습니다': 7345, '다가': 2183, '취소': 10758, '해도': 11942, '국내': 1122, '약사

In [13]:
X = all_OTA['content']
y = all_OTA['sentiment']

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

X_train_morphs = []
for doc in X_train:
    # 형태소 분석
    morphs = okt.morphs(doc, stem=True)
    # 불용어 제거
    morphs = [word for word in morphs if not word in stopwords]
    # 분석한 형태소를 다시 공백으로 결합하여 저장
    X_train_morphs.append(' '.join(morphs))

In [16]:
# TF-IDF 변환
tfidf_vectorizer = TfidfVectorizer(vocabulary=vector.vocabulary_)
X_train = tfidf_vectorizer.fit_transform(X_train_morphs)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

X_test_morphs = []
for doc in X_test:
    # 형태소 분석
    morphs = okt.morphs(doc, stem=True)
    # 불용어 제거
    morphs = [word for word in morphs if not word in stopwords]
    # 분석한 형태소를 다시 공백으로 결합하여 저장
    X_test_morphs.append(' '.join(morphs))

In [18]:
# TF-IDF 변환
tfidf_vectorizer = TfidfVectorizer(vocabulary=vector.vocabulary_)
X_test = tfidf_vectorizer.fit_transform(X_test_morphs)

In [19]:
X_test

<7527x12426 sparse matrix of type '<class 'numpy.float64'>'
	with 60682 stored elements in Compressed Sparse Row format>

In [20]:
print(X_train.shape)
print(X_test.shape)

(17560, 12426)
(7527, 12426)


### SVM

In [21]:
param_grid = {
    'C' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'sigmoid'],
    "gamma": ["scale", "auto"],
}

In [22]:
from sklearn.svm import SVC

svm = SVC()

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
# 지표를 하나만 설정할 경우
from sklearn.model_selection import cross_val_score
# 지표를 하나 이상 설정할 경우
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 모델의 최적의 하이퍼파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 군집화
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth



# ARIMA (시계열 예측)
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm


# 시간 측정을 위한 시간 모듈
import datetime

# 주식정보
from pandas_datareader import data

# 형태소 벡터를 생성하기 위한 라이브러리
from sklearn.feature_extraction.text import CountVectorizer
# 형태소 벡터를 학습 벡터로 변환한다.
from sklearn.feature_extraction.text import TfidfTransformer


# 데이터 수집
import requests
from bs4 import BeautifulSoup
import re
import time
import os
import json

# 한국어 형태소 분석
from konlpy.tag import Okt, Hannanum, Kkma, Mecab, Komoran

# 저장
import pickle

In [24]:
from sklearn.model_selection import GridSearchCV

svm_grid = GridSearchCV(
    svm,
    param_grid,
    cv=5,
    scoring="accuracy",
    verbose=1,
    n_jobs=-1,
)

In [25]:
svm_grid.fit(X_train, y_train)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'sigmoid']},
             scoring='accuracy', verbose=1)

In [26]:
# Print the best parameters and the best score
print("Best parameters:", svm_grid.best_params_)
print("Best score:", svm_grid.best_score_)

Best parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
Best score: 0.9378701594533029


In [27]:
from sklearn.metrics import accuracy_score

# Test the model with the test dataset
y_pred = svm_grid.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9449980071741729


In [28]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.86      0.89      1870
           1       0.96      0.97      0.96      5657

    accuracy                           0.94      7527
   macro avg       0.93      0.92      0.92      7527
weighted avg       0.94      0.94      0.94      7527



In [29]:
import joblib

# 모델 저장
joblib.dump(svm_grid.best_estimator_, '/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/감성분석모델/svm_model.pkl')

['/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/감성분석모델/svm_model.pkl']

In [30]:
import pickle

# vector.vocabulary_ 저장
with open("/content/drive/MyDrive/리뷰데이터 분석/데이터셋/데이터함/감성분석모델/vocabulary.pkl", "wb") as f:
    pickle.dump(vector.vocabulary_, f)

### 로지스틱 회귀분석

In [31]:
from sklearn.linear_model import LogisticRegression

param_grid = {
    'C' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet', 'none']
}


In [32]:
logistic_grid = GridSearchCV(
    LogisticRegression(),
    param_grid,
    cv=5,
    scoring="accuracy",
    verbose=1,
    n_jobs=-1,
)

In [33]:
logistic_grid.fit(X_train, y_train)

Fitting 5 folds for each of 28 candidates, totalling 140 fits


GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none']},
             scoring='accuracy', verbose=1)

In [34]:
# Print the best parameters and the best score
print("Best parameters:", logistic_grid.best_params_)
print("Best score:", logistic_grid.best_score_)

Best parameters: {'C': 10, 'penalty': 'l2'}
Best score: 0.9322323462414579


In [35]:
# Test the model with the test dataset
y_pred = logistic_grid.predict(X_test)

In [36]:
# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9394180948585094


In [37]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.85      0.87      1870
           1       0.95      0.97      0.96      5657

    accuracy                           0.94      7527
   macro avg       0.93      0.91      0.92      7527
weighted avg       0.94      0.94      0.94      7527



### KNN

In [38]:
from sklearn.neighbors import KNeighborsClassifier

param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

In [39]:
knn_grid = GridSearchCV(
    KNeighborsClassifier(),
    param_grid,
    cv=5,
    scoring="accuracy",
    verbose=1,
    n_jobs=-1,
)

In [40]:
knn_grid.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [3, 5, 7, 9, 11], 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy', verbose=1)

In [41]:
# Print the best parameters and the best score
print("Best parameters:", knn_grid.best_params_)
print("Best score:", knn_grid.best_score_)

Best parameters: {'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}
Best score: 0.7933940774487471


In [42]:
# Test the model with the test dataset
y_pred = knn_grid.predict(X_test)

In [43]:
# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7985917364155706


In [44]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.22      0.36      1870
           1       0.79      0.99      0.88      5657

    accuracy                           0.80      7527
   macro avg       0.83      0.61      0.62      7527
weighted avg       0.81      0.80      0.75      7527



### 나이브베이즈

In [45]:
from sklearn.naive_bayes import GaussianNB

param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}


In [46]:
nb_grid = GridSearchCV(
    GaussianNB(),
    param_grid,
    cv=5,
    scoring="accuracy",
    verbose=1,
    n_jobs=-1,
)


In [47]:
nb_grid.fit(X_train.toarray(), y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


GridSearchCV(cv=5, estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]},
             scoring='accuracy', verbose=1)

In [48]:
# Print the best parameters and the best score
print("Best parameters:", nb_grid.best_params_)
print("Best score:", nb_grid.best_score_)

Best parameters: {'var_smoothing': 1e-05}
Best score: 0.8652050113895215


In [49]:
# Test the model with the test dataset
y_pred = nb_grid.predict(X_test.toarray())

In [50]:
# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8666135246446127


In [51]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.67      0.71      1870
           1       0.89      0.93      0.91      5657

    accuracy                           0.87      7527
   macro avg       0.83      0.80      0.81      7527
weighted avg       0.86      0.87      0.86      7527

